In [2]:

pip install pyaudio


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install SpeechRecognition

   ---------------------------------------- 0.0/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/32.8 MB 2.6 MB/s eta 0:00:13
    --------------------------------------- 0.7/32.8 MB 7.6 MB/s eta 0:00:05
   - -------------------------------------- 1.3/32.8 MB 9.1 MB/s eta 0:00:04
   - -------------------------------------- 1.5/32.8 MB 8.2 MB/s eta 0:00:04
   -- ------------------------------------- 1.9/32.8 MB 7.9 MB/s eta 0:00:04
   -- ------------------------------------- 2.1/32.8 MB 7.6 MB/s eta 0:00:05
   -- ------------------------------------- 2.4/32.8 MB 7.2 MB/s eta 0:00:05
   --- ------------------------------------ 2.6/32.8 MB 7.0 MB/s eta 0:00:05
   --- ------------------------------------ 2.9/32.8 MB 7.0 MB/s eta 0:00:05
   --- ------------------------------------ 3.1/32.8 MB 6.8 MB/s eta 0:00:05
   ---- ----------------------------------- 3.5/32.8 MB 6.7 MB/s eta 0:00:05
   ---- ----------------------------------- 3.8/32.8 MB 6.5 MB/s eta 0:00:05
   ---

In [3]:
pip install pyttsx3 

   ---------------------------------------- 0.0/210.6 kB ? eta -:--:--
   --------------------------------- ------ 174.1/210.6 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 210.6/210.6 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow


     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/377.0 MB ? eta -:--:--
   ---------------------------------------- 0.4/377.0 MB 23.9 MB/s eta 0:00:16
   ---------------------------------------- 0.7/377.0 MB 11.6 MB/s eta 0:00:33
   ---------------------------------------- 0.8/377.0 MB 7.5 MB/s eta 0:00:51
   ---------------------------------------- 0.9/377.0 MB 6.2 MB/s eta 0:01:01
   ---------------------------------------- 1.4/377.0 MB 6.8 MB/s eta 0:00:56
   ---------------------------------------- 1.4/377.0 MB 6.8 MB/s eta 0:00:56
   ---------------------------------------- 1.7/377.0 MB 5.9 MB/s eta 0:01:04
   ---------------------------------------- 2.2/377.0 MB 6.2 MB/s eta 0:01:01
   ---------------------------------------- 2.5/377.0 MB 6.2 MB/s eta 0:01:01
   ---------------------------------------- 2.8/377.0 MB 6.2 MB/s eta 0:0

In [11]:
import json
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
import speech_recognition as sr
import pyttsx3
import sys

# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Load intents file
with open('dataset.json') as file:
    data = json.load(file)

lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

# Preprocess data
for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Prepare training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

random.shuffle(training)

# Separate the features and labels
train_x = [np.array(tr[0]) for tr in training]
train_y = [np.array(tr[1]) for tr in training]

train_x = np.array(train_x)
train_y = np.array(train_y)

# Build and train model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')

# Load model
model = load_model('chatbot_model.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    if not ints:
        return "Sorry, I didn't understand that."
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = get_response(ints, data)
    return res

def get_speech_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print(f"User said: {text}")
            return text
        except sr.UnknownValueError:
            print("Sorry, I did not understand that.")
            return ""
        except sr.RequestError:
            print("Sorry, my speech service is down.")
            return ""

def speak_text(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def main():
    while True:
        input_type = input("Do you want to use text or speech input? (text/speech): ").strip().lower()
        if input_type == "text":
            while True:
                message = input("You: ")
                if message.lower() == "quit":
                    sys.exit("User requested to quit.")
                response = chatbot_response(message)
                print(f"Bot: {response}")
        elif input_type == "speech":
            while True:
                print("Say 'quit' to stop the conversation.")
                message = get_speech_input()
                if message.lower() == "quit":
                    sys.exit("User requested to quit.")
                response = chatbot_response(message)
                print(f"Bot: {response}")
                speak_text(response)
        else:
            print("Invalid input. Please enter 'text' or 'speech'.")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vimer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vimer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 814us/step - accuracy: 0.0133 - loss: 4.6729 
Epoch 2/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step - accuracy: 0.0467 - loss: 4.3758
Epoch 3/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0755 - loss: 4.1870 
Epoch 4/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0788 - loss: 4.0650
Epoch 5/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0882 - loss: 3.8445 
Epoch 6/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step - accuracy: 0.1052 - loss: 3.7463
Epoch 7/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.1219 - loss: 3.5548
Epoch 8/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1488 - loss: 3.5422
Epoch 9/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1892 - loss: 3.3094  
Epoch 10/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2381 - loss: 3.2004
Epoch 11/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2544 - loss: 2.9769
Epoch 12/200
124/124 ━━━━━━━

Do you want to use text or speech input? (text/speech):  speech


Say 'quit' to stop the conversation.
Listening...
Sorry, I did not understand that.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Bot: Great to see you. How do you feel currently?
Say 'quit' to stop the conversation.
Listening...
User said: I am fine
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Oh nice to meet you. Tell me how was your week?
Say 'quit' to stop the conversation.
Listening...
User said: what was good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Sorry, I didn't understand that.
Say 'quit' to stop the conversation.
Listening...
User said: it was good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Good evening. How has your day been?
Say 'quit' to stop the conversation.
Listening...
User said: better
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Can you elaborate on that?
Say 'quit' to stop the conversation.
Listening...
User said: I am happy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Did something happen which made you feel this way?
Say 'quit' to stop the conversation.
Listening...
User sai

SystemExit: User requested to quit.